In [ ]:
import sys
sys.path.insert(0, './openpose_torch/python')
import cv2
import model
import util
from hand import Hand
from body import Body
import matplotlib.pyplot as plt
import copy
import numpy as np

%matplotlib inline
%config InlineBackend.figure_format = 'retina'


In [ ]:
# import pretrained model
body_estimation = Body('./openpose_torch/model/body_pose_model.pth')
hand_estimation = Hand('./openpose_torch/model/hand_pose_model.pth')

In [ ]:
# sample image
sample_image = cv2.imread("./openpose_torch/images/me.jpeg")

In [ ]:
# reverse channel(because opencv always does)
sample_image_reverse = sample_image[:,:,::-1]

# show
plt.imshow(sample_image_reverse)

In [ ]:
# body estimation
candidate, subset = body_estimation(sample_image)

In [ ]:
canvas = np.zeros(sample_image_reverse.shape)

canvas = util.draw_bodypose(canvas, candidate, subset)

plt.imshow(canvas[:,:,::-1])

write video

In [ ]:
file_name = "./openpose_torch/images/sample_video.MOV"

video_capture = cv2.VideoCapture()

video_capture.open(file_name)

fps = video_capture.get(cv2.CAP_PROP_FPS)

max_second = 3

seconds = 0

stick_post_img_list = []

while seconds < max_second:
    seconds += 1/fps
    
    has_frames, frame = video_capture.read()
    
    if not has_frames:
        break
        
    candidate, subset = body_estimation(frame)
        
    canvas = np.zeros(frame.shape)
    
    canvas = util.draw_bodypose(canvas, candidate, subset)
    
    stick_post_img_list.append(canvas)
    
video_capture.release()

In [ ]:
output_file_name = "output.avi"

fourcc = cv2.VideoWriter_fourcc('M','J','P','G')
# fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')


output_frame_size = frame.shape[:2]

video_writer = cv2.VideoWriter(output_file_name, fourcc, fps, output_frame_size, True)

for stick_frame in stick_post_img_list:
    video_writer.write(stick_frame.astype("uint8"))
    
video_writer.release()

In [ ]:
video_writer.release()
video_writer = None